# Anitapolis single L1 TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data of Anitápolis.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os
import pandas as pd

from fatiando.vis import mpl

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

### Auxiliary functions

In [3]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing Anitapolis TFA data

In [4]:
data = pd.read_csv('anitapolis_decimated_tfa_residual_cut.txt', skipinitialspace=True, delim_whitespace=True)

In [5]:
data['GPSALT'] = - data['GPSALT'] + 800

In [6]:
mask = (data['GPSALT'].get_values()<0.)
data = data[mask]

In [7]:
xp = data['X'].get_values()
yp = data['Y'].get_values()
zp = data['GPSALT'].get_values()
alt = data['ALTURA'].get_values()
dobs = data['mag_res1'].get_values()

### Parameters of the initial model

In [8]:
M = 20 # number of vertices per prism
L = 6 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = -21
decs = -11

int_min = 11.
int_max = 20.
intensity = np.linspace(int_min, int_max, 10)

# depth to the top, thickness and radius
z0_min = 0.
z0_max = 360.
z0 = np.linspace(z0_min, z0_max, 10)
dz = 630.
r = 700.
x0 = 6921000.
y0 = 688000.

# main field
inc, dec = [-37.05, -18.17]

In [9]:
z0

array([  0.,  40.,  80., 120., 160., 200., 240., 280., 320., 360.])

In [10]:
intensity

array([11., 12., 13., 14., 15., 16., 17., 18., 19., 20.])

### Outcropping parameters

In [11]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Limits

In [12]:
# limits for parameters in meters
rmin = 10.
rmax = 1200.
y0min = 687000.
y0max = 699000.
x0min = 6916000.
x0max = 6923000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Derivatives

In [13]:
# variation for derivatives
deltax = 0.01*np.max(100.)
deltay = 0.01*np.max(100.)
deltar = 0.01*np.max(100.)
deltaz = 0.01*np.max(100.)

delta = np.array([deltax, deltay, deltar, deltaz])

### Regularization parameters

In [14]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-4  # adjacent radial distances within each prism
a2 = 1.0e-3   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4    # vertically adjacent origins
a6 = 1.0e-8   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-5     # zero order Tikhonov on thickness of each prism

alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

### Folder to save the results

In [15]:
foldername = '43485-new'

In [16]:
itmax = 30
itmax_marq = 10
tol = 1.0e-3     # stop criterion

### Inversion

In [17]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        print 'inversion: %d  top: %d  intensity: %d' % (j*z0.size + k, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz, 1
        )
        inversion_results.append([m_est, phi_list, model_list, dobs - d_fit])

inversion: 0  top: 0  intensity: 11
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 5.30007e+01  fin obj.: 4.13634e+01
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 4.13634e+01  fin obj.: 3.70875e+01
it:  2   it_marq:  0   lambda: 1e-01   init obj.: 3.70875e+01  fin obj.: 3.84295e+01
it:  2   it_marq:  1   lambda: 1e+00   init obj.: 3.70875e+01  fin obj.: 3.49013e+01
it:  3   it_marq:  0   lambda: 1e-01   init obj.: 3.49013e+01  fin obj.: 4.03408e+01
it:  3   it_marq:  1   lambda: 1e+00   init obj.: 3.49013e+01  fin obj.: 3.29275e+01
it:  4   it_marq:  0   lambda: 1e-01   init obj.: 3.29275e+01  fin obj.: 4.29005e+01
it:  4   it_marq:  1   lambda: 1e+00   init obj.: 3.29275e+01  fin obj.: 3.19974e+01
it:  5   it_marq:  0   lambda: 1e-01   init obj.: 3.19974e+01  fin obj.: 4.28632e+01
it:  5   it_marq:  1   lambda: 1e+00   init obj.: 3.19974e+01  fin obj.: 3.11982e+01
it:  6   it_marq:  0   lambda: 1e-01   init obj.: 3.11982e+01  fin obj.: 4.37800e+01
it:  6   it_marq:  1   lambda

# Results

In [18]:
# output of inversion
inversion = dict()

inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['origin'] = [x0, y0]
inversion['observed_data'] = dobs
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['initial_r'] = r
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity
inversion['results'] = inversion_results

### Saving results

In [19]:
if foldername == '':
    mypath = 'l1-tfa-inversion/multiple-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = 'l1-tfa-inversion/multiple-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

In [20]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)

In [21]:
mypath

'l1-tfa-inversion/multiple-43485-new'